In [1]:
# Mirror Project Readdme
from IPython.display import display, Markdown
with open('README.md', 'r') as f:
    content = f.read()
display(Markdown(content))

# iAudit INDaaS (Independence as a Service)
## Private, Independent Audits of Alternative Inter-Cloud Replication Deployments 

Customers of modern cloud services deploy replicas of their applications 
in order to improve reliability and performance. These replicas may be spread 
across multiple providers, such as Amazon AWS and Microsoft Azure.

Shared vulnerabilities (both hardware and software) of providers can lead to 
cascading failures, since the benefits of redundancy are lost. Despite this risk 
to consumers, cloud providers are reluctant to share vulnerability information 
with other providers. How can we let consumers achieve sufficient redundancy 
in their application deployments, while also recognizing the business concerns
of cloud providers?

Introducing  **iAudit**. 

**iAudit** is a system for computing the weighted shared dependency score
between multiple cloud providers, which does not require any of the providers
to disclose their vulnerabilities to the customer or competing providers. Lower
score indicates higher independence. Higher independence scores indicate deployment schemes with higher  application reliability.

There are two components to the system. This repository contains Python
code for the second, with influence from the [Java](https://github.com/ennanzhai/auditor) implementation.

1. Generation of provider dependency graphs
2. Compute dependency scores using a Weighted Private Set Intersection Cardinality Protocol (Vaidya and Clifton)

Part of a senior project at Yale University.

### iAudit Team
- Project Lead: [Ennan Zhai, PhD](http://www.cs.yale.edu/homes/zhai-ennan/)
- Advisor: [Profesor Avi Silberschatz](http://codex.cs.yale.edu/avi/)
- Students: William Dower, [Cameron Yick](www.cameronyick.us)


### Single Node Example

Show that protocol works when doing ring pass on a single computer


In [2]:
import iaudit

In [3]:
# Dummy data
'''
 A cutset is a list of vulernabilities that would break the application on a given node.
 A node can have multiple cutsets.
 Each cutset has a weight, indicating its significance. Higher score = more significance.
 In these cells, we setup
'''

strings = {}

strings['A'] = \
"""
<cutsets>
    <cutset items="DNS1" weight="10"/>
    <cutset items="Agg1, Agg2" weight="6"/>
</cutsets>
"""

strings['B'] = \
"""
<cutsets>
    <cutset items="DNS1" weight="10"/>
    <cutset items="Agg3" weight="6"/>
    <cutset items="DNS2" weight="4"/>
</cutsets>
"""

In [4]:
# Unweighted Example
cutsetsA = iaudit.string_to_cutsets(strings['A'])
cutsetsB = iaudit.string_to_cutsets(strings['B'])

In [5]:
cutsets = [list(cutsetlist.cutsets) for cutsetlist in [cutsetsA, cutsetsB]]

In [6]:
# Because of the weights, we won't have true set interesection.
# for c in cutsets.cutsets:
#     print c

In [7]:
# cutsets[0]

## Adding Encryption


First, we try out an RSA based method, like the Sefasi tool.


In [10]:
# For demo, these variables are hard coded into notebook. In practice, will live as config files on each node.
# Same across all nodes
# Use sample numbers from http://www.di-mgt.com.au/rsa_alg.html
pub_config = {}
# # pub_config['encryption'] = {
# #     'p': 47,
# #     'q': 13
# # }
pub_config['encryption'] = {
    'p': 240392533562968913097477233624108603431,
    'q': 320671053885185848480799823061733369081
}

# Private Configuration
pvt_config = {}
pvt_config['encryption'] = {
    'e': 60664108018938273705006436868609408115043521529170409238549489718805733807407
}


Let's differentiate master from worker code.

### Master
1. Generate p, q given modlen
2. Send p, q to all workers

### Worker
1. Opens read config file and reads p, q, keyLen
2. Each worker computes a local 'e'
3. Worker tells master it is "done"




In [11]:
'''
RSA Directions:
The modulus **n** must be the product of two primes.
The public exponent **e** must be odd and larger than 1.

In case of a private key, the following equations must apply:

- e != 1
- p*q = n
- e*d = 1 mod (p-1)(q-1)
- p*u = 1 mod q

# order of arguments for RSA.Construct
1. RSA modulus (n).
2. Public exponent (e).
3. Private exponent (d). Only required if the key is private.
4. First factor of n (p). Optional.
5. Second factor of n (q). Optional.
6. CRT coefficient, (1/p) mod q (u). Optional.
'''

def make_private_key(public_config, private_config):
    '''
    Args:
        public_config (dict): Contains public modulus p, q
        private_config (dict): Contains private exponent e
    Returns:
        An RSA key object (`_RSAobj`).
    
    '''
    # public key contains public modulus
    # in production, use a local public key file
    p = long(public_config['encryption']['p'])
    q = long(public_config['encryption']['q'])
    # private key contains private exponent
    e = long(private_config['encryption']['e'])

    # Enforce constraint that p must be bigger.
    # http://crypto.stackexchange.com/questions/18084/in-rsa-why-does-p-have-to-be-bigger-than-q-where-n-p-times-q
    if p < q:
        p, q = q, p
        
    # in practice, not needed 
    n = p*q
    phi = (p - 1)*(q - 1)
    d = pubkey.inverse(e, phi)
    rsaInit = (n, e, d, p, q)
    #     rsaInit = (n, e, d)
    #     print "n {}, e {}, d {}, p {}, q {}".format(n, e, d, p, q)
    return RSA.construct(rsaInit)

- e*d = 1 mod (p-1)(q-1)
- p*u = 1 mod q


In [12]:
# Setup the crypto 
pvtKey = make_private_key(pub_config, pvt_config)
pubKey = pvtKey.publickey()

In [13]:
# With our weak example, we must use a short message 
# See: http://0x90909090.blogspot.com/2015/10/playing-with-rsa-and-python.html
# We must assert that the length of our key is 
import mmh3

message = mmh3.hash128('foo') # 128 bit signed int
message

168394135621993849475852668931176482145L

In [14]:
encrypted = pubKey.encrypt(message, None) # second parameter is ignored
print encrypted

(45892529642076698608147709124482775227112778692088602152411741677410112216709L,)


In [15]:
pvtKey.decrypt(encrypted)

168394135621993849475852668931176482145L

Let's check out a sample local workflow.


In [16]:
list(cutsetsA.cutsets)

[]

In [17]:
# first time only! 
items = list(cutsetsA.cutsets)
random.shuffle(items)

# encrypt each time
encrypted = [pubKey.encrypt(item, None) for item in items ]

print encrypted


[]


#### Multi node Demo: Does encryption still work with multiple parties?



In [18]:
# Random configuration files
pubConfig  = generate_public_config(1024)
pvtConfigs = [generate_private_config(pubConfig, 256, seed) for seed in range(2)]

In [19]:
pvtConfigs

[{'encryption': {'e': 111793196543967404139194827996419963236210979610743141064269745943111491389389L},
  'keyBits': 256,
  'type': 'private'},
 {'encryption': {'e': 24311105965938388548830698200525867024144360738597094831162501213063826691451L},
  'keyBits': 256,
  'type': 'private'}]

In [20]:
# Random keys
# Setup the crypto 
pvtKeys = [make_private_key(pubConfig, pvtConfig) for pvtConfig in pvtConfigs] # actually aren't needed!
pubKeys = [pvt.publickey() for pvt in pvtKeys]

In [21]:
# all messsages
encryptedA = cutsets[0]
encryptedB = cutsets[1]

In [22]:
# now, make the encrypted sets
for key in pubKeys:
    encryptedA = [key.encrypt(message, None)[0] for message in encryptedA]
    encryptedB = [key.encrypt(message, None)[0] for message in encryptedB]
    
    # Shuffle in between passes
    random.shuffle(encryptedA)
    random.shuffle(encryptedB)

In [23]:
# test weighted cardinality -> correct!
print len(set(encryptedA) & set(encryptedB))
print len(encryptedA) + len(encryptedB)

10
36


If our data is purely numeric (e.g. with Sefasi they just had IP addresses which have fixed size, this is good enough. However, our list of dependencies may contain strings of arbitrary length. Thus, we want to try a different encryption scheme that can overcome these limits.


SRA calculator [here](https://asecuritysite.com/encryption/comm2)

Why RSA has message length restriction [here](http://stackoverflow.com/questions/10061626/message-length-restriction-in-rsa)



Would it still be sensible to get this working with SRA protocol, knowing that we limit all cutset lengths to be less than a bounded number of characters?


What about "Fast Private Set Intersection Cardinality" with the help of Bloom Filters?

http://www.ics.uci.edu/~gts/paps/psi-ca.pdf

### Use AES to overcome item length limits



In [24]:
# import base64
# from Crypto.Cipher import AES
# from Crypto import Random
# import os

# import os
# import hashlib


In [25]:
# for i in range(10):
#     print crypto.decrypt(encrypted)

In [26]:
# x1 = Cipher("key2")
# x2 = Cipher("key1")

In [27]:
# a1 = x2.encrypt(x1.encrypt("4a"))
# # need to reset counter states 
# a2 = x1.encrypt(x2.encrypt("4a"))

In [28]:
# x1.decrypt(x2.decrypt(a1))

NameError: name 'x1' is not defined

In [ ]:
# x2.decrypt(a1)


In [ ]:
# x1.decrypt(a1)

In [ ]:
# x1.decrypt(a2)

In [ ]:
# import Crypto.Cipher.AES
# import Crypto.Util.Counter

# key1 = "01234ABCDEF56789" # test key
# key2 = "123343cswEABCDEF"

# # replace this with a RANDOMLY GENERATED VALUE, and send this with the ciphertext!
# # let every member participate with the same iv
# iv1 = "2100000000000001" 
# iv2 = "0000000000005678" 

# message = "foo bar bear bake"
    
# ctr1 = Crypto.Util.Counter.new(128, initial_value=long(iv1.encode("hex"), 16))
# ctr2 = Crypto.Util.Counter.new(128, initial_value=long(iv1.encode("hex"), 16))

# cipher1 = Crypto.Cipher.AES.new(key1, Crypto.Cipher.AES.MODE_CTR, counter=ctr1)
# cipher2 = Crypto.Cipher.AES.new(key2, Crypto.Cipher.AES.MODE_CTR, counter=ctr2)


In [ ]:
# def cipher(key, iv, message):
#     ctr = Crypto.Util.Counter.new(128, initial_value=long(iv.encode("hex"), 16)) # can we reuse this?
#     _cipher = Crypto.Cipher.AES.new(key, Crypto.Cipher.AES.MODE_CTR, counter=ctr)
#     return _cipher.encrypt(message)


In [ ]:
# a1 = cipher2.encrypt(cipher1.encrypt(message))

In [ ]:
# cipher(key2, iv2, "cake")

In [ ]:
# cipher(key1, iv1, "cake")

In [ ]:
# at the very end, need to provide all sides with counter reset to the same initial values.
# ctr1 = Crypto.Util.Counter.new(128, initial_value=long(iv.encode("hex"), 16))
# ctr2 = Crypto.Util.Counter.new(128, initial_value=long(iv.encode("hex"), 16))
# cipher1 = Crypto.Cipher.AES.new(key1, Crypto.Cipher.AES.MODE_CTR, counter=ctr1)
# cipher2 = Crypto.Cipher.AES.new(key2, Crypto.Cipher.AES.MODE_CTR, counter=ctr2)

# a2 = cipher1.encrypt(cipher2.encrypt(message))


In [ ]:
# cipher2.decrypt(a1)

Readings: Implementing the Mental Poker Scheme with SRA


- https://en.wikipedia.org/wiki/Mental_poker
- http://people.csail.mit.edu/rivest/ShamirRivestAdleman-MentalPoker.pdf
- https://link.springer.com/chapter/10.1007%2F3-540-39799-X_10

General Crypto
- https://9d0df72831e4b345bb93-4b37fd03e6af34f2323bb971f72f0c0d.ssl.cf5.rackcdn.com/Crypto101.pdf
- Python has Pycrypto (unmaintained), as well as Pycrytodome. See cryptography.io
- https://security.stackexchange.com/questions/33434/rsa-maximum-bytes-to-encrypt-comparison-to-aes-in-terms-of-security
- [Basic Python RSA Example](http://stackoverflow.com/questions/30056762/rsa-encryption-and-decryption-in-python)
- [Pycryptodome](https://pycryptodome.readthedocs.io/en/latest/src/examples.html#encrypt-data-with-aes)


Commutative Crypto
- http://crypto.stackexchange.com/questions/11083/can-we-decrypt-in-this-order-when-the-message-is-encrypted-twice/11084#11084
- http://crypto.stackexchange.com/questions/41328/is-there-an-encryption-method-where-the-order-of-decryption-is-irrelevant-to-the

Why we can't reuse the [counter](https://www.dlitz.net/software/pycrypto/api/current/Crypto.Util.Counter-module.html)

- https://security.stackexchange.com/questions/63188/whats-the-point-of-the-nonce-in-ctr-mode

- https://security.stackexchange.com/questions/27776/block-chaining-modes-to-avoid
- http://crypto.stackexchange.com/questions/2377/deterministic-nonces-in-ctr-mode
- http://crypto.stackexchange.com/questions/11361/how-good-is-using-aes-ctr-mode-with-initial-counter-as-0
- http://crypto.stackexchange.com/questions/23993/ivs-for-different-aes-encryption-modes

Python AES CTR Examples

- http://stackoverflow.com/questions/14714968/pycrypto-aes-ctr-implementation?rq=1
- http://stackoverflow.com/questions/11656045/pycrypto-incrementing-ctr-mode
- http://stackoverflow.com/questions/21440506/aes-ctr-implementation
- http://stackoverflow.com/questions/3154998/pycrypto-problem-using-aesctr
- http://stackoverflow.com/questions/12691168/how-aes-in-ctr-works-for-python-with-pycrypto?answertab=oldest#tab-top

Links

- Commutative? https://www.experts-exchange.com/questions/22807749/Commutative-encryption-for-secret-key-exchange.html
- About crypto and poker: https://www.cs.purdue.edu/homes/ninghui/courses/Fall05/lectures/355_Fall05_lect25.pdf
- can bloom filters help to make this faster [paper](http://www.utdallas.edu/~muratk/publications/pakdd09.pdf)

Is this work similar to zero-knowledge proof projects?
- http://crypto.stackexchange.com/questions/1363/software-implementation-of-a-commutative-cipher
- http://zk-ssh.cms.ac/
- [Privacy-preserving set intersection protocol](http://fnp.sourceforge.net/)


### Multi Node Example

Extending program to work when managing communication between multiple computers


Checking if port is in use
- http://stackoverflow.com/questions/19196105/python-how-to-check-if-a-network-port-is-open-on-linux
Check if port is listening
- http://serverfault.com/questions/309052/check-if-port-is-open-or-closed-on-a-linux-server


In [ ]:
# Use hostname as the salt to the hashing function as a prototype
from iaudit import network 
network.hostname()

In [31]:
# let us create 
workingDirectory =  os.getcwdu()
configPath = os.path.join(workingDirectory, "iaudit-config.json")
print configPath

C:\Users\cyick\Projects\iaudit-python\iaudit-config.json


In [ ]:
# decide to use http because don't need to calculate message lengths in advance.

In [ ]:
# use sockets or http?
# http://www.binarytides.com/python-socket-programming-tutorial/
import socket                                 
import sys                                    
                                              
HOST, PORT = config['masterHost'].split(':')                    
PORT = int(PORT) # master port  
                                              
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
print 'Socket created'                        
                                              
try:                                          
    s.bind((HOST, PORT))                      
except socket.error , msg:                    
    print 'Bind failed. Error Code : ' + str(msg[0]) + ' Message ' + msg[1]
    sys.exit()                                
                                              
print 'Socket bind complete'                  
                                              
s.listen(5) # how many connect requests allowed  to be queued

print 'Socket now listening'                  
           
nReceived = 0

#now keep talking with the client             
while True:                                      
    #wait to accept a connection - blocking call 
    conn, addr = s.accept() 

    nReceived  += 1                  
    print 'Connected with ' + addr[0] + ':' +  str(addr[1])                           
    data = conn.recv(1024)                    
    reply = 'Received...' + data                    
    if nReceived == numWorkers:
        break
    else:
        continue
                
print("Cleaning up!")                                       
conn.close()                                  
s.close()  